# Bases de datos vectoriales

Hemos visto en el post de [embeddings](https://www.maximofn.com/embeddings) que los embeddings son una forma de representar palabras en un espacio vectorial. En este post vamos a ver cómo podemos almacenar esos embeddings en bases de datos vectoriales y cómo podemos hacer consultas sobre ellas.

Cuando tenemos una consulta, podemos crear el embedding de la consulta, buscar en la base de datos vectorial los embeddings que más se parezcan a la consulta y devolver los documentos que correspondan a esos embeddings o una explicaión sobre esos documentos.

![vector database](http://maximofn.com/wp-content/uploads/2024/02/vector_database.svg)

Es decir, vamos a generar una base de datos de información, vamos a crear embeddings de esa información y la vamos a guardar en una base de datos vectorial. Luego cuando un usuario haga una consulta, convertiremos la consulta a embeddings, buscaremos en la base de datos los embeddings con mayor similitud y devolveremos los documentos que correspondan a esos embeddings.

Además de los documentos, en la base de datos se puede guardar información adicional que llamaremos metadata. Por ejemplo, si estamos trabajando con un conjunto de noticias, podemos guardar el título, la fecha, el autor, etc. de la noticia.

## Chroma

En este post vamos a ver [crhoma](https://www.trychroma.com/), ya que es la base de [datos vectorial más usada](https://blog.langchain.dev/langchain-state-of-ai-2023/#most-used-vectorstores), como se puede ver en este reporte del [langchain state of ai 2023](https://blog.langchain.dev/langchain-state-of-ai-2023).

![Most Used Vectorstores](https://blog.langchain.dev/content/images/size/w1000/2023/12/Top-vectorstores--1-.png)

## Instalación

De modo que para instalar chroma con conda hay que hacer

```bash
conda install conda-forge::chromadb
```

O si se quiere instalar con pip

```bash
pip install chromadb
```

## Uso rápido

Para una aplicación rápida, primero importamos chroma

In [1]:
import chromadb

A continuación creamos un cliente de chroma

In [2]:
chroma_client = chromadb.Client()

Creamos una colección. Una colección es el lugar donde se guardarán los embeddings, los embeddings y la metadata.

In [3]:
collection = chroma_client.create_collection(name="my_collection")

Como vemos sale un mensaje indicando que no se ha introducido una función de embeddings y por lo tanto usará por defecto `all-MiniLM-L6-v2`, que es similar al modelo `paraphrase-MiniLM-L6-v2` que usamos en el post de [embeddings](https://maximofn.com/embeddings/).

Más adelante veremos esto, pero podemos elegir cómo vamos a generar los embeddings.

Ahora añadimos documentos, ids y metadata a la colección

In [4]:
collection.add(
    documents=["This is a python docs", "This is JavaScript docs"],
    metadatas=[{"source": "Python source"}, {"source": "JavaScript source"}],
    ids=["id1", "id2"]
)

Ahora podemos hacer una consulta

In [5]:
results = collection.query(
    query_texts=["This is a query of Python"],
    n_results=2
)

In [6]:
results

{'ids': [['id1', 'id2']],
 'distances': [[0.6205940246582031, 1.4631636142730713]],
 'metadatas': [[{'source': 'Python source'}, {'source': 'JavaScript source'}]],
 'embeddings': None,
 'documents': [['This is a python docs', 'This is JavaScript docs']],
 'uris': None,
 'data': None}

Como vemos la distancia al id1 es menor a la distancia al id2, por lo que parece que el documento 1 es más apropiado para responder la consulta

## Bases de datos persistentes

La base de datos que hemos creado antes es temporal, en cuanto cerremos el notebook desaparecerá. Por lo que para crear una base de datos persistente hay que pasarle a chroma el path donde guardarla

Primero vamos a crear la carpeta donde guardar la base de datos

In [7]:
from pathlib import Path

chroma_path = Path("chromadb")
chroma_path.mkdir(exist_ok=True)

Ahora creamos un cliente en la carpeta que hemos creado

In [8]:
chroma_client_persistent = chromadb.PersistentClient(path = str(chroma_path))

## Colecciones

### Crear colecciones

A la hora de crear una colección hay que especificar un nombre. El nombre tiene que tener las siguientes consideraciones:

 * La longitud del nombre debe tener entre 3 y 63 caracteres.
 * El nombre debe comenzar y terminar con una letra minúscula o un dígito y puede contener puntos, guiones y guiones bajos en el medio.
 * El nombre no debe contener dos puntos consecutivos.
 * El nombre no debe ser una dirección IP válida.

También podemos darle una función de embedding. En caso de no darle una usará por defecto la función `all-MiniLM-L6-v2`

In [9]:
collection = chroma_client.create_collection(name="my_other_collection")

Como se puede ver, se ha creado una segunda colección para el mismo cliente `chroma_client`, por lo que para un solo cliente podemos tener varias colecciones.

### Recuperar colecciones

Si queremos recuperar una colección de un cliente lo podemos hacer con el método `get_collection`

In [10]:
collection = chroma_client.get_collection(name = "my_collection")

### Recuperar o crear colecciones

Podemos obtener colecciones y que en caso de que no existan las cree con el método `get_or_create_collection`

In [13]:
collection = chroma_client.get_or_create_collection(name = "my_tird_collection")

### Borrar colecciones

Podemos borrar una colección con el método `delete_collection`

In [14]:
chroma_client.delete_collection(name="my_tird_collection")

### Obtener items de las colecciones

Podemos obtener los 10 primeros items de la colección con el método `peek`

In [15]:
collection = chroma_client.get_collection(name = "my_collection")
collection.peek()

{'ids': ['id1', 'id2'],
 'embeddings': [[-0.06924048811197281,
   0.061624377965927124,
   -0.090973399579525,
   0.013923337683081627,
   0.006247623357921839,
   -0.1078396588563919,
   -0.012472339905798435,
   0.03485661745071411,
   -0.06300634145736694,
   -0.00880391988903284,
   0.06879935413599014,
   0.0564003586769104,
   0.07040536403656006,
   -0.020754728466272354,
   -0.04048658534884453,
   -0.006666888482868671,
   -0.0953674241900444,
   0.049781784415245056,
   0.021780474111437798,
   -0.06344643980264664,
   0.06119797006249428,
   0.0834411084651947,
   -0.034758951514959335,
   0.0029120452236384153,
   0.021940510720014572,
   -0.043226245790719986,
   -0.022839635610580444,
   0.0401647612452507,
   0.02025778777897358,
   -0.014062674716114998,
   -0.0007076433976180851,
   0.07914472371339798,
   0.07875147461891174,
   0.05343058705329895,
   0.028390051797032356,
   0.05079597234725952,
   0.04415654018521309,
   -0.041151899844408035,
   -0.016470894217491

En este caso solo se han obtenido dos, porque nuestra colección solo tiene dos documentos

Si se quiere obtener otra cantidad de items se puede especificar con el argumento `limit`

In [16]:
collection.peek(limit=1)

{'ids': ['id1'],
 'embeddings': [[-0.06924048811197281,
   0.061624377965927124,
   -0.090973399579525,
   0.013923337683081627,
   0.006247623357921839,
   -0.1078396588563919,
   -0.012472339905798435,
   0.03485661745071411,
   -0.06300634145736694,
   -0.00880391988903284,
   0.06879935413599014,
   0.0564003586769104,
   0.07040536403656006,
   -0.020754728466272354,
   -0.04048658534884453,
   -0.006666888482868671,
   -0.0953674241900444,
   0.049781784415245056,
   0.021780474111437798,
   -0.06344643980264664,
   0.06119797006249428,
   0.0834411084651947,
   -0.034758951514959335,
   0.0029120452236384153,
   0.021940510720014572,
   -0.043226245790719986,
   -0.022839635610580444,
   0.0401647612452507,
   0.02025778777897358,
   -0.014062674716114998,
   -0.0007076433976180851,
   0.07914472371339798,
   0.07875147461891174,
   0.05343058705329895,
   0.028390051797032356,
   0.05079597234725952,
   0.04415654018521309,
   -0.041151899844408035,
   -0.01647089421749115,
   

### Obtener el número total de items de las colecciones

Podemos obtener el número total de items de la colección con el método `count`

In [17]:
collection.count()

2

### Cambiar la función de similitud

Antes, cuando hicimos una consulta obtuvimos la similitud de los embeddings con nuestra consulta, ya que por defecto en una colección se usa la función de distancia, pero podemos especificar qué función de similitud queremos usar. Las posiilidades son

 * Squared L2 (`l2`)
 * Inner product (`ip`)
 * Cosine similarity (`cosine`)

En el post [Medida de similitud entre embeddings](http://maximofn.com/embeddings-similarity/) vimos L2 y cosine similarity, por si quieres profundizar en ellas.

Por lo que podemos crear colecciones con otra función de similitud con el argumento `metadata={"hnsw:space": <funtion>}`

In [18]:
collection = chroma_client.create_collection(name="colection_cosine", metadata={"hnsw:space": "cosine"})

### Añadir datos a la colección

Vamos a volver a ver los datos que tenemos en la colección con el método `peek`

In [19]:
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

Como vemos está vacío, eso es porque la última colección que hemos creado ha sido la de la función de similitud `cosine`, pero no le hemos añadido datos. Veamos como es así obteniendo el nombre de la colección

In [20]:
collection.name

'colection_cosine'

Por lo que nos volvemos a traer la primera colección que hemos creado, que sí le hemos introducido datos

In [22]:
collection = chroma_client.get_collection(name = "my_collection")

Ahora ya podemos añadir datos a la collección con el método `add`

In [23]:
collection.add(
    documents=["This is a Mojo docs", "This is Rust docs"],
    metadatas=[{"source": "Mojo source"}, {"source": "Rust source"}],
    ids=["id3", "id4"]
)

Como se puede ver los `ID`s son consecutivos y no tienen el mismo valor que ya tenían antes, ya que los `ID`s tienen que ser únicos. Si intentamos repetir lo anterior, nos idiciará que ya existían datos con esas `ID`s

In [27]:
collection.add(
    documents=["This is a Pytorch docs", "This is TensorRT docs"],
    metadatas=[{"source": "Pytorch source"}, {"source": "TensorRT source"}],
    ids=["id3", "id4"]
)

Insert of existing embedding ID: id3
Insert of existing embedding ID: id4
Add of existing embedding ID: id3
Add of existing embedding ID: id4


Veamos los datos de la colección

In [28]:
collection.peek()

{'ids': ['id1', 'id2', 'id3', 'id4'],
 'embeddings': [[-0.06924048811197281,
   0.061624377965927124,
   -0.090973399579525,
   0.013923337683081627,
   0.006247623357921839,
   -0.1078396588563919,
   -0.012472339905798435,
   0.03485661745071411,
   -0.06300634145736694,
   -0.00880391988903284,
   0.06879935413599014,
   0.0564003586769104,
   0.07040536403656006,
   -0.020754728466272354,
   -0.04048658534884453,
   -0.006666888482868671,
   -0.0953674241900444,
   0.049781784415245056,
   0.021780474111437798,
   -0.06344643980264664,
   0.06119797006249428,
   0.0834411084651947,
   -0.034758951514959335,
   0.0029120452236384153,
   0.021940510720014572,
   -0.043226245790719986,
   -0.022839635610580444,
   0.0401647612452507,
   0.02025778777897358,
   -0.014062674716114998,
   -0.0007076433976180851,
   0.07914472371339798,
   0.07875147461891174,
   0.05343058705329895,
   0.028390051797032356,
   0.05079597234725952,
   0.04415654018521309,
   -0.041151899844408035,
   -0.0

Como vemos se han mantenido los contenidos originales de `ID3` e `ID4`

### Consultas